In [169]:
import json
import sys,os
%load_ext autoreload
%autoreload 2
sys.path.append(os.path.abspath('..'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [170]:
import tensorflow as tf
import numpy as np
from bunch import Bunch
from model.gpt import gpt
from model.gpt import sample
from distributed_encoder import gpt_encoder

In [184]:
config = Bunch({})
config.n_ctx = 512
config.n_embd = 20
config.n_vocab = 1000
config.n_head = 2
config.n_embd = 10
config.n_layer = 1

np_x = np.array([[2,5,4,6,7,3], [2,7,9,10,11,3]]).astype(np.int32)

def top_k_logits(logits, k):
    if k == 0:
        # no truncation
        return logits

    def _top_k():
        values, _ = tf.nn.top_k(logits, k=k)
        min_values = values[:, -1, tf.newaxis]
        return tf.where(
            logits < min_values,
            tf.ones_like(logits, dtype=logits.dtype) * -1e10,
            logits,
        )
    values, _ = tf.nn.top_k(logits, k=k)
    min_values = values[:, -1, tf.newaxis]
#     return values, min_values, logits
    return tf.cond(
       tf.equal(k, 0),
       lambda: logits,
       lambda: _top_k(),
    )

graph = tf.Graph()
with graph.as_default():
    features = {}
    features['input_ids'] = tf.constant(np_x)
    features['past'] = None
    model = gpt_encoder.gpt_encoder(config, features, [], 
                                    tf.estimator.ModeKeys.TRAIN, 
                                    "", reuse=tf.AUTO_REUSE)
    logits = model.get_sequence_output_logits()
    resp = top_k_logits(logits[:,0], 10)
    
#     model.build_model(config, x, None, scope='model', reuse=tf.AUTO_REUSE)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    s = sess.run(resp)
    print(s.shape)

(2, 1, 2, 2, 6, 5)
(2, 1000)


In [206]:
graph = tf.Graph()
import time
with graph.as_default():
    
    input_ids = tf.placeholder(tf.int32, [None, None], name='input_ids')
    
    result = sample.sample_sequence(gpt_encoder.gpt_encoder, config, 
                200, start_token=None, 
                batch_size=5, 
#                 context=tf.constant(np.array([[0,1], [0,2]]).astype(np.int32)), 
                context = input_ids,
                temperature=1, top_k=10)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    start = time.time()
    s = (sess.run(result, feed_dict={
        input_ids:np.array([[0,1,2,3,4], [0,2,3,4,5]])
    }))
    print(s, time.time()-start)
    start = time.time()
#     s = (sess.run(result))
#     print(s, time.time()-start)

(?, 1, 2, 2, ?, 5)
(?, 1, 2, 2, ?, 5)
{'logits': array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1702917 , 0.16923714, 0.16711274, 0.14681551, 0.16808064,
        0.19813028, 0.147789  , 0.17816603, 0.17283107, 0.18569289,
        0.2133118 , 0.24055089, 0.23212756, 0.17792062, 0.16522826,
        0.2546756 , 0.17508416, 0.24439278, 0.16722316, 0.19870318,
        0.21101728, 0.17866698, 0.18448557, 0.15058343, 0.17433824,
        0.14285442, 0.1500796 , 0.15595567, 0.19464596, 0.15855123,
        0.15251201, 0.2313006 , 0.19689348, 0.16852157, 0.17276806,
        0.19716509, 0.14104508, 0.14188062, 0.14443657, 0.1583072 ,
        0.15649562, 0.15696244, 0.15960234, 0.19457208, 0.17940626,
        0.14595081, 0.17476676, 0.16401511, 0.18440592, 0.15136689,
        0.16147162, 0.1574808 , 0.17872457, 0.17289576, 0.1667252 ,
        0.1993473 , 0.16827649, 0.16021575, 0.17661972, 0.15498729,
        0.17402618, 0.22665107, 0.19187896, 0.19031852, 0.16141292,

In [205]:
s['logits'].shape

(2, 205)

In [173]:
graph = tf.Graph()
with graph.as_default():
    sess = tf.Session()
    logits = tf.constant(np.random.random((2,100)).astype(np.float32))
    sample1 = tf.constant(([[0],[1]]))
    
    
    
    l = tf.gather_nd(logits, sample1)
    print(sess.run([l]))

[array([[0.7612247 , 0.52393895, 0.71110415, 0.26772088, 0.90555954,
        0.9578271 , 0.74923176, 0.5355166 , 0.8057869 , 0.95264626,
        0.985974  , 0.05059365, 0.97953206, 0.15599953, 0.88650906,
        0.8713196 , 0.5319897 , 0.08941194, 0.9837196 , 0.49263823,
        0.84202725, 0.21526779, 0.5155082 , 0.670111  , 0.08703706,
        0.8281144 , 0.30705503, 0.01450889, 0.33066344, 0.25443307,
        0.86935556, 0.04914412, 0.5602702 , 0.8231789 , 0.9982192 ,
        0.5526849 , 0.35354188, 0.9384927 , 0.73802286, 0.39589548,
        0.13057972, 0.66236556, 0.64637226, 0.884436  , 0.01557355,
        0.00772093, 0.15658706, 0.08196006, 0.37628266, 0.51681846,
        0.45165926, 0.67655814, 0.27004886, 0.3292241 , 0.28690296,
        0.6918035 , 0.09440008, 0.8832438 , 0.6760844 , 0.7875932 ,
        0.3907096 , 0.9485808 , 0.21338293, 0.28876367, 0.57318854,
        0.07479472, 0.9935976 , 0.40469816, 0.00154824, 0.618636  ,
        0.79971427, 0.1092016 , 0.6062513 , 0.2

In [166]:
s = tf.constant(np.random.random((1, 6, 3)))
l = tf.constant(np.random.random((2*3*4, 3)))
s_l = tf.reshape(l, [3, 2, -1])
s_l_1 = tf.reshape(l, [6, -1])
# s_p = tf.reshape(s, [1,2,3, 3])
# pp = tf.concat(tf.split(s, 3,axis=1), axis=-1)
# print(pp)
resp = (sess.run([s_l, s_l_1]))
resp[0]

array([[[0.18870372, 0.51728041, 0.79367226, 0.79581814, 0.03477913,
         0.97816948, 0.97826842, 0.33662379, 0.62231756, 0.08760684,
         0.33744362, 0.59310832],
        [0.16122543, 0.67664858, 0.51616022, 0.89368816, 0.06049976,
         0.20845996, 0.86946212, 0.46144452, 0.22209935, 0.40889158,
         0.54539087, 0.31221015]],

       [[0.4212259 , 0.13559438, 0.00700644, 0.59372448, 0.65645485,
         0.68347948, 0.3122973 , 0.19347655, 0.99004872, 0.02311294,
         0.51892345, 0.28276387],
        [0.84719068, 0.86786945, 0.61192696, 0.19027599, 0.6423064 ,
         0.72969829, 0.95019531, 0.67915218, 0.46796741, 0.0031278 ,
         0.09840903, 0.6669415 ]],

       [[0.88592617, 0.64015442, 0.79774302, 0.82703768, 0.89772438,
         0.63176179, 0.01658025, 0.42579049, 0.0722788 , 0.03270506,
         0.18701915, 0.43889291],
        [0.7481108 , 0.80221182, 0.83814664, 0.56706657, 0.45391693,
         0.18632884, 0.35135013, 0.10334133, 0.60213403, 0.51715691

In [167]:
resp[1]

array([[0.18870372, 0.51728041, 0.79367226, 0.79581814, 0.03477913,
        0.97816948, 0.97826842, 0.33662379, 0.62231756, 0.08760684,
        0.33744362, 0.59310832],
       [0.16122543, 0.67664858, 0.51616022, 0.89368816, 0.06049976,
        0.20845996, 0.86946212, 0.46144452, 0.22209935, 0.40889158,
        0.54539087, 0.31221015],
       [0.4212259 , 0.13559438, 0.00700644, 0.59372448, 0.65645485,
        0.68347948, 0.3122973 , 0.19347655, 0.99004872, 0.02311294,
        0.51892345, 0.28276387],
       [0.84719068, 0.86786945, 0.61192696, 0.19027599, 0.6423064 ,
        0.72969829, 0.95019531, 0.67915218, 0.46796741, 0.0031278 ,
        0.09840903, 0.6669415 ],
       [0.88592617, 0.64015442, 0.79774302, 0.82703768, 0.89772438,
        0.63176179, 0.01658025, 0.42579049, 0.0722788 , 0.03270506,
        0.18701915, 0.43889291],
       [0.7481108 , 0.80221182, 0.83814664, 0.56706657, 0.45391693,
        0.18632884, 0.35135013, 0.10334133, 0.60213403, 0.51715691,
        0.41264404,

In [168]:
sess.run(l)


array([[0.18870372, 0.51728041, 0.79367226],
       [0.79581814, 0.03477913, 0.97816948],
       [0.97826842, 0.33662379, 0.62231756],
       [0.08760684, 0.33744362, 0.59310832],
       [0.16122543, 0.67664858, 0.51616022],
       [0.89368816, 0.06049976, 0.20845996],
       [0.86946212, 0.46144452, 0.22209935],
       [0.40889158, 0.54539087, 0.31221015],
       [0.4212259 , 0.13559438, 0.00700644],
       [0.59372448, 0.65645485, 0.68347948],
       [0.3122973 , 0.19347655, 0.99004872],
       [0.02311294, 0.51892345, 0.28276387],
       [0.84719068, 0.86786945, 0.61192696],
       [0.19027599, 0.6423064 , 0.72969829],
       [0.95019531, 0.67915218, 0.46796741],
       [0.0031278 , 0.09840903, 0.6669415 ],
       [0.88592617, 0.64015442, 0.79774302],
       [0.82703768, 0.89772438, 0.63176179],
       [0.01658025, 0.42579049, 0.0722788 ],
       [0.03270506, 0.18701915, 0.43889291],
       [0.7481108 , 0.80221182, 0.83814664],
       [0.56706657, 0.45391693, 0.18632884],
       [0.

In [152]:
resp[0]

array([[[[0.26561798, 0.07820186, 0.56343956],
         [0.15781025, 0.52902424, 0.04970787],
         [0.80937254, 0.71862167, 0.98620838]],

        [[0.45159752, 0.11242172, 0.96958232],
         [0.75665836, 0.03443459, 0.36699389],
         [0.160791  , 0.46020058, 0.71200601]]]])

In [110]:
my_jaccard_score('领券减元日本正品超薄款运动内衣女无钢圈薄款无痕文胸背心胸罩', '薄款无痕文胸背心胸罩', 2)

0.3333333333333333

In [118]:
from fuzzywuzzy import fuzz
import jieba
print(fuzz.ratio("领券 减元 日本 正品 超薄 款 运动 内衣 女 无 钢圈", 
                 "薄款 无痕 文胸 背心 胸罩"))

28


In [116]:
" ".join(jieba.cut('领券减元日本正品超薄款运动内衣女无钢圈薄款无痕文胸背心胸罩'))," ".join(jieba.cut('薄款无痕文胸背心胸罩'))

('领券 减元 日本 正品 超薄 款 运动 内衣 女 无 钢圈', '薄款 无痕 文胸 背心 胸罩')